In [1]:
#!/usr/bin/env python
# coding: utf-8

# In[1]:


import warnings
warnings.filterwarnings('ignore')
import pandas as pd
import liana as li
import anndata
import scanpy as sc
from openproblems import tasks
import os 


# In[2]:


# Install required packages.
import os
import torch
os.environ['TORCH'] = torch.__version__
print(torch.__version__)

# !pip install -q torch-scatter -f https://data.pyg.org/whl/torch-${TORCH}.html
# !pip install -q torch-sparse -f https://data.pyg.org/whl/torch-${TORCH}.html
# !pip install -q git+https://github.com/pyg-team/pytorch_geometric.git

# Helper function for visualization.
import matplotlib.pyplot as plt
from sklearn.manifold import TSNE
from utils import *


# In[3]:


import argparse
import os.path as osp

import torch
import torch.nn.functional as F

from torch_geometric.datasets import Entities
from torch_geometric.nn import FastRGCNConv, RGCNConv, GCNConv, InnerProductDecoder, GAE, VGAE
from torch_geometric.utils import k_hop_subgraph

from torch_geometric.datasets import Planetoid
from torch_geometric.data.data import Data
from torch_geometric.transforms import NormalizeFeatures


from sklearn.cluster import KMeans
import matplotlib.pyplot as plt

import pandas as pd
import numpy as np

from sklearn.metrics.pairwise import cosine_similarity
from sklearn.cluster import SpectralClustering
import random
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt

from sklearn.metrics.pairwise import cosine_similarity

from scipy.io import mmread

from scipy.stats import wilcoxon
import anndata
import scanpy as sc

1.12.0+cu102
1.12.0+cu102


In [48]:
def make_nodes_interactions(matrix):
    matrix.index = [str(i).upper() for i in matrix.index.tolist()]



    index = matrix.index.tolist()

    matrix = matrix.fillna(0)

    adata = anndata.AnnData(matrix.transpose())

    sc.pp.filter_genes(adata, min_cells=10)
    sc.pp.normalize_total(adata)
    sc.pp.neighbors(adata)
    sc.tl.leiden(adata)
    meta = pd.DataFrame({"cell":adata.obs["leiden"].index.tolist(),"labels":adata.obs["leiden"].tolist()})


    # In[ ]:


    meta.index = meta["cell"].tolist()


    # In[ ]:


    matrix = matrix.loc[adata.var.index.tolist()]


    # In[ ]:


    #sc.tl.rank_genes_groups(adata, 'labels')
    cell_groups = meta['labels'].unique().tolist()
    matrix_list = {}
    for i in cell_groups:
        cells = meta[meta["labels"]==i].index.tolist()
        temp_matrix = matrix[cells]
        matrix_list[i] = (temp_matrix.mean(axis=1)[temp_matrix.mean(axis=1) > 0].index.tolist())

    cell_type_df = matrix_list


    # In[23]:


    #mean_expression = np.mean(matrix.values[matrix.values != 0])
    nodes = pd.DataFrame({"category":[],"identifier":[]})


    # In[24]:


    LR_nodes = pd.read_csv("/h/soemily/GAT/data/LR_database/OmniPath_nodes.csv",index_col=0)
    Omnipath_network = pd.read_csv("/h/soemily/GAT/data/LR_database/OmniPath_interactions.csv",index_col=0)


    # In[25]:


    ligands = LR_nodes[LR_nodes["category"]=="Ligand"]["identifier"].tolist()
    receptors = LR_nodes[LR_nodes["category"]=="Receptor"]["identifier"].tolist()

    ligand_list = []
    receptor_list = []
    new_cell_df = {}
    for i in cell_type_df.keys():
        ligand_list.extend(list(set(ligands) & set(cell_type_df[i])))
        receptor_list.extend(list(set(receptors) & set(cell_type_df[i])))
        new_cell_df[i] = [list(set(ligands) & set(cell_type_df[i])),list(set(receptors) & set(cell_type_df[i]))]


    # In[27]:


    for i in new_cell_df.keys():
        new_cell_df[i][0] = [j+"_Ligand" for j in new_cell_df[i][0]]
        new_cell_df[i][1] = [j+"_Receptor" for j in new_cell_df[i][1]]


    # In[28]:


    ligand_list = list(set(ligand_list))
    receptor_list = list(set(receptor_list))


    # In[29]:


    ligand_list = [i+"_Ligand" for i in ligand_list]
    receptor_list = [i+"_Receptor" for i in receptor_list]


    # In[30]:


    nodes = pd.concat([nodes,pd.DataFrame({"category":["Cell Group"] * len(list(cell_type_df.keys())), "identifier":list(cell_type_df.keys())})])
    nodes = pd.concat([nodes,pd.DataFrame({"category":["Ligand"] * len(ligand_list), "identifier":ligand_list})])
    nodes = pd.concat([nodes,pd.DataFrame({"category":["Receptor"] * len(receptor_list), "identifier":receptor_list})])


    # In[31]:


    new_identifier = [row["identifier"] + "_" + row["category"] for index,row in LR_nodes.iterrows()]


    # In[32]:


    LR_nodes["identifier"] = new_identifier


    # In[33]:


    nodes["Id"] = range(0,nodes.shape[0])
    nodes = nodes[["Id","category","identifier"]]


    # In[34]:


    nodes.index = nodes.index.astype('int')
    nodes["Id"] = nodes["Id"].astype('int')


    # In[35]:


    meta.index = meta["cell"]


    # In[36]:


    interactions = pd.DataFrame({"Src":[],"Dst":[],"Weight":[],"edge_type":[]})


    # In[37]:


    LR_nodes.index = LR_nodes["Id"].tolist()


    # In[38]:


    Omnipath_network["Src"] = LR_nodes.loc[Omnipath_network["Src"].tolist()]["identifier"].tolist()
    Omnipath_network["Dst"] = LR_nodes.loc[Omnipath_network["Dst"].tolist()]["identifier"].tolist()


    # In[39]:


    source_list = []
    dest_list = []
    weight_list = []
    edge_type_list = []
    for i in new_cell_df.keys():
        source_list.extend([i] * (len(new_cell_df[i][0]) + len(new_cell_df[i][1])))
        dest_list.extend(new_cell_df[i][0])
        dest_list.extend(new_cell_df[i][1])
        weight_list.extend([1] * (len(new_cell_df[i][0]) + len(new_cell_df[i][1])))
        edge_type_list.extend([1] * (len(new_cell_df[i][0]) + len(new_cell_df[i][1])))


    # In[40]:


    interactions["Src"] = source_list
    interactions["Dst"] = dest_list
    interactions["Weight"] = weight_list
    interactions["edge_type"] = edge_type_list


    # In[41]:


    nodes.index = nodes["identifier"].tolist()


    # In[42]:


    nodes = nodes.drop_duplicates("identifier")
    nodes["Id"] = range(0,nodes.shape[0])


    # In[43]:


    interactions["Src"] = nodes.loc[interactions["Src"].tolist()]["Id"].tolist()
    interactions["Dst"] = nodes.loc[interactions["Dst"].tolist()]["Id"].tolist()

    LR_nodes = pd.read_csv("/h/soemily/GAT/data/LR_database/OmniPath_nodes.csv",index_col=0)
    Omnipath_network = pd.read_csv("/h/soemily/GAT/data/LR_database/OmniPath_interactions.csv",index_col=0)


    # In[55]:


    new_identifier = [row["identifier"] + "_" + row["category"] for index,row in LR_nodes.iterrows()]


    # In[56]:


    LR_nodes["identifier"] = new_identifier


    # In[57]:


    Omnipath_network["Src"] = LR_nodes.loc[Omnipath_network["Src"].tolist()]["identifier"].tolist()
    Omnipath_network["Dst"] = LR_nodes.loc[Omnipath_network["Dst"].tolist()]["identifier"].tolist()


    # In[58]:


    LR_nodes = LR_nodes[(LR_nodes["identifier"].isin(ligand_list)) | (LR_nodes["identifier"].isin(receptor_list))]


    # In[59]:


    Omnipath_network = Omnipath_network[(Omnipath_network["Src"].isin(LR_nodes["identifier"].tolist())) & (Omnipath_network["Dst"].isin(LR_nodes["identifier"].tolist()))]


    # In[60]:


    LR_nodes["Id"] = range(0,LR_nodes.shape[0])


    # In[61]:


    LR_nodes.index = LR_nodes["identifier"].tolist()
    Omnipath_network["Src"] = LR_nodes.loc[Omnipath_network["Src"].tolist()]["Id"].tolist()
    Omnipath_network["Dst"] = LR_nodes.loc[Omnipath_network["Dst"].tolist()]["Id"].tolist()


    # In[62]:


    LR_nodes = LR_nodes[(LR_nodes["Id"].isin(Omnipath_network["Src"].tolist())) | (LR_nodes["Id"].isin(Omnipath_network["Dst"].tolist()))]


    # In[63]:


    LR_nodes.index = LR_nodes["Id"].tolist()
    Omnipath_network["Src"] = LR_nodes.loc[Omnipath_network["Src"].tolist()]["identifier"].tolist()
    Omnipath_network["Dst"] = LR_nodes.loc[Omnipath_network["Dst"].tolist()]["identifier"].tolist()


    # In[64]:


    LR_nodes.index = LR_nodes["identifier"].tolist()
    LR_nodes["Id"] = range(0,LR_nodes.shape[0])
    Omnipath_network["Src"] = LR_nodes.loc[Omnipath_network["Src"].tolist()]["Id"].tolist()
    Omnipath_network["Dst"] = LR_nodes.loc[Omnipath_network["Dst"].tolist()]["Id"].tolist()

    return matrix,meta,nodes,interactions,LR_nodes,Omnipath_network


In [49]:

from torch_geometric.nn import GATConv
class GAT(torch.nn.Module):
    def __init__(self,data,num_classes=3):
        super(GAT, self).__init__()
        self.hid = 3
        self.in_head = 3
        self.out_head = 3
        
        
        self.conv1 = GATConv(data.x.shape[1], self.hid, heads=self.in_head, dropout=0.6)
        self.conv2 = GATConv(self.hid*self.in_head,num_classes, concat=False,
                             heads=self.out_head, dropout=0.6)

    def forward(self, data):
        x, edge_index = data.x, data.edge_index
                
        x = F.dropout(x, p=0.6, training=self.training)
        x = self.conv1(x, edge_index)
        x = F.elu(x)
        x = F.dropout(x, p=0.6, training=self.training)
        x = self.conv2(x, edge_index)
        x = F.elu(x)
        x = F.logsigmoid(x)
        return x
    
    
    
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device = "cpu"
    


# # Get Omnipath embedding

# In[54]:




# In[65]:


from torch_geometric.nn import Node2Vec
import os.path as osp
import torch
import matplotlib.pyplot as plt
from sklearn.manifold import TSNE
from torch_geometric.datasets import Planetoid
from tqdm.notebook import tqdm


# In[66]:


def train(model,loader,optimizer):
    model.train()
    total_loss = 0
    for pos_rw, neg_rw in (loader):
        optimizer.zero_grad()
        loss = model.loss(pos_rw.to(device), neg_rw.to(device))
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    return total_loss / len(loader)


# In[67]:


@torch.no_grad()
def test():
    model.eval()
    z = model()
    acc = model.test(z[data.train_mask], data.y[data.train_mask],
                     z[data.test_mask], data.y[data.test_mask],
                     max_iter=150)
    return acc


# In[68]:


In [50]:
def get_Omnipath_embeddings(nodes,interactions):
    Omnipath_data,Omnipath_nodes,Omnipath_interactions = make_dataset(nodes,interactions,first=False,pathway_encode=False)
    node_info = pd.DataFrame(np.zeros((Omnipath_nodes.shape[0],Omnipath_nodes.shape[0])),index=Omnipath_nodes["identifier"].tolist(),columns=Omnipath_nodes["identifier"].tolist())

    temp_identifiers = [i.split("_")[0] for i in Omnipath_nodes["identifier"].tolist()]

    complexes = pd.read_csv("/h/soemily/GAT/data/data/LR_database/complexes.csv")
    complexes = complexes[complexes["member"].isin(temp_identifiers)]

    temp_nodes = Omnipath_nodes.copy()
    temp_nodes.index = temp_identifiers
    temp_nodes = temp_nodes[~temp_nodes.index.duplicated(keep='first')]

    complexes["member"] = temp_nodes.loc[complexes["member"].tolist()]["identifier"].tolist()

    group_complex = complexes.groupby("complex").agg(list)

    group_complex.index=range(0,group_complex.shape[0])

    for index,row in group_complex.iterrows():
        node_info.loc[list(set(row["member"])),list(set(row["member"]))] = index

    # for i in group_complex["member"].tolist():
    #     node_info.loc[list(set(i)),list(set(i))] = 1

    pathways = pd.read_csv("/h/soemily/GAT/data/kegg_pathways.csv",index_col=0)
    pathways = pathways[pathways["genesymbol"].isin(temp_identifiers)]
    pathways["genesymbol"] = temp_nodes.loc[pathways["genesymbol"].tolist()]["identifier"].tolist()
    group_pathway = pathways.groupby("pathway").agg(list)

    group_pathway.index=range(0,group_pathway.shape[0])

    for index,row in group_pathway.iterrows():
        node_info.loc[list(set(row["genesymbol"])),list(set(row["genesymbol"]))] += index

    # for i in group_pathway["genesymbol"].tolist():
    #     node_info.loc[list(set(i)),list(set(i))] += 1

    truth_info = pd.DataFrame(np.zeros((Omnipath_nodes.shape[0],Omnipath_nodes.shape[0])),index=Omnipath_nodes["identifier"].tolist(),columns=Omnipath_nodes["identifier"].tolist())

    Omnipath_nodes.index = Omnipath_nodes["Id"].tolist()

    ident_interactions = Omnipath_interactions.copy()
    ident_interactions["Src"] = Omnipath_nodes.loc[ident_interactions["Src"].tolist()]["identifier"].tolist()
    ident_interactions["Dst"] = Omnipath_nodes.loc[ident_interactions["Dst"].tolist()]["identifier"].tolist()

    for index,row in ident_interactions.iterrows():
        truth_info.loc[row["Src"],row["Dst"]] = 1

    ligands = Omnipath_nodes[Omnipath_nodes["category"]=="Ligand"]["identifier"].tolist()
    receptors = Omnipath_nodes[Omnipath_nodes["category"]=="Receptor"]["identifier"].tolist()
    #truth_info = truth_info.loc[ligands,receptors]
    truth_info = torch.Tensor(truth_info.values).to(device)


    ligands = Omnipath_nodes[Omnipath_nodes["category"]=="Ligand"]["Id"].tolist()
    receptors = Omnipath_nodes[Omnipath_nodes["category"]=="Receptor"]["Id"].tolist()


    ident_interactions = ident_interactions.drop_duplicates("Src")
    ident_interactions = ident_interactions.drop_duplicates("Dst")

    ident_interactions.index = range(0,ident_interactions.shape[0])

    truth_list = []
    for i in Omnipath_nodes["identifier"].tolist():
        if "Ligand" in i:
            if i in ident_interactions["Src"].tolist():
                #truth_list.append(ident_interactions[ident_interactions["Src"]==i].index.tolist()[0] + 1)
                truth_list.append(1)
            else:
                truth_list.append(0)
        if "Receptor" in i:
            if i in ident_interactions["Dst"].tolist():
                #truth_list.append(ident_interactions[ident_interactions["Dst"]==i].index.tolist()[0] + 1)
                truth_list.append(1)
            else:
                truth_list.append(0)

    node_info.values[np.where(np.isnan(node_info.values))] = 0
    node_info.values[np.where(np.isinf(node_info.values))] = 0

    Omnipath_data.x = torch.Tensor(node_info.values)

    Omnipath_data.y = torch.Tensor(truth_list).type(torch.LongTensor)
    #Omnipath_data.y = truth_info

    Omnipath_nodes.index = Omnipath_nodes["Id"].tolist()

    Omnipath_interactions["Src"] = [Omnipath_nodes.loc[i]["identifier"] for i in Omnipath_interactions["Src"].tolist()]
    Omnipath_interactions["Dst"] = [Omnipath_nodes.loc[i]["identifier"] for i in Omnipath_interactions["Dst"].tolist()]

    #edge_weights = [max(node_info.loc[i,j],node_info.loc[j,i]) for i,j in zip(Omnipath_interactions["Src"].tolist(),Omnipath_interactions["Dst"].tolist())]
    edge_weights = [1 for i,j in zip(Omnipath_interactions["Src"].tolist(),Omnipath_interactions["Dst"].tolist())]
    data = Omnipath_data
    model = Node2Vec(data.edge_index, embedding_dim=2, walk_length=40,
         context_size=40, walks_per_node=10,
         num_negative_samples=1, p=1, q=1, sparse=True).to(device)

    loader = model.loader(batch_size=2, shuffle=True, num_workers=4)
    optimizer = torch.optim.SparseAdam(list(model.parameters()), lr=0.01)
    
    
    for epoch in range(1):
        loss = train(model,loader,optimizer)
        #acc = test()
#         if epoch % 10 == 10:
#             print(f'Epoch: {epoch:02d}, Loss: {loss:.4f}')
    model.eval()
    z = model(torch.arange(data.num_nodes)).detach()

    ligand_ids = Omnipath_nodes[Omnipath_nodes["category"].str.contains("Ligand")]["Id"].tolist()
    receptor_ids = Omnipath_nodes[Omnipath_nodes["category"].str.contains("Receptor")]["Id"].tolist()

    ligand_embeddings = z[ligand_ids,:]
    receptor_embeddings = z[receptor_ids,:]

    total_embeddings = torch.inner(ligand_embeddings,receptor_embeddings)

    total_embeddings_df = pd.DataFrame(total_embeddings.numpy(),index=Omnipath_nodes[Omnipath_nodes["category"].str.contains("Ligand")]["identifier"].tolist(),columns=Omnipath_nodes[Omnipath_nodes["category"].str.contains("Receptor")]["identifier"].tolist())
    return total_embeddings_df


# In[69]:


In [77]:
df_list = []
def get_cell_LR_embeddings(matrix,meta,nodes,interactions,total_embeddings_df,Omnipath_nodes,Omnipath_interactions):
    device = 'cpu'
    truth_info = pd.DataFrame(np.zeros((Omnipath_nodes.shape[0],Omnipath_nodes.shape[0])),index=Omnipath_nodes["identifier"].tolist(),columns=Omnipath_nodes["identifier"].tolist())

    #Omnipath_nodes.index = Omnipath_nodes["Id"].tolist()

    ident_interactions = Omnipath_interactions.copy()
    ident_interactions["Src"] = Omnipath_nodes.loc[ident_interactions["Src"].tolist()]["identifier"].tolist()
    ident_interactions["Dst"] = Omnipath_nodes.loc[ident_interactions["Dst"].tolist()]["identifier"].tolist()

    for index,row in ident_interactions.iterrows():
        truth_info.loc[row["Src"],row["Dst"]] = 1

    ligands = Omnipath_nodes[Omnipath_nodes["category"]=="Ligand"]["identifier"].tolist()
    receptors = Omnipath_nodes[Omnipath_nodes["category"]=="Receptor"]["identifier"].tolist()
    #truth_info = truth_info.loc[ligands,receptors]
    truth_info = torch.Tensor(truth_info.values).to(device)

    
    cell_LR_data,cell_LR_nodes,cell_LR_ints = make_dataset(nodes,interactions,first=False,pathway_encode=False)

    full_matrix = pd.DataFrame(np.zeros((cell_LR_nodes.shape[0],cell_LR_nodes.shape[0])),index=cell_LR_nodes["identifier"].tolist(),columns=cell_LR_nodes["identifier"].tolist())

    ligands = Omnipath_nodes[Omnipath_nodes["category"]=="Ligand"]["Id"].tolist()
    receptors = Omnipath_nodes[Omnipath_nodes["category"]=="Receptor"]["Id"].tolist()

    total_out_df = total_embeddings_df

    gene_mean = matrix.mean(axis=1)

    Omnipath_nodes.index = [i.split("_")[0] for i in Omnipath_nodes["identifier"].tolist()]

    Omnipath_nodes = Omnipath_nodes.loc[~Omnipath_nodes.index.duplicated(),:].copy()
    
    gene_mean = gene_mean.loc[Omnipath_nodes.index.tolist()]

    gene_mean.index = Omnipath_nodes["identifier"].tolist()

    ligands = [i for i in gene_mean.index.tolist() if "Ligand" in i]
    receptors = [i for i in gene_mean.index.tolist() if "Receptor" in i]
    ligands = list(set(full_matrix.index.tolist()) & set(ligands))
    receptors = list(set(full_matrix.index.tolist()) & set(receptors))

    for i,j in zip(ligands,receptors):
        if (i in gene_mean.index.tolist()) and (j in gene_mean.index.tolist()):
            full_matrix.loc[i,j] = gene_mean.loc[i]*gene_mean.loc[j]

    ligands = list(set(ligands) & set(total_out_df.index.tolist()))
    receptors = list(set(receptors) & set(total_out_df.columns.tolist()))

    for i,j in zip(ligands,receptors):
        full_matrix.loc[i,j] += total_out_df.loc[i,j]


    #full_matrix.loc[true_df.index.tolist(),true_df.columns.tolist()] = true_df.values

    cell_groups = meta["labels"].unique().tolist()

    ligands = cell_LR_nodes[cell_LR_nodes["category"]=="Ligand"]["identifier"].tolist()
    ligands = [i.split("_")[0] for i in ligands]
    ligand_matrix = matrix.loc[ligands]

    ligand_matrix = ligand_matrix[~ligand_matrix.index.duplicated(keep='first')]


    mean_dict = {}
    for i in cell_groups:
        cells = meta[meta["labels"]==i]["cell"].tolist()
        mean_dict[i] = ligand_matrix[cells].mean(axis=1)

    full_matrix = full_matrix[~full_matrix.index.duplicated(keep='first')]
    full_matrix = full_matrix.loc[:,~full_matrix.columns.duplicated()].copy()


    for i in mean_dict.keys():
        temp_index = [i+"_Ligand" for i in mean_dict[i].index.tolist()]
        full_matrix.loc[i,temp_index] = mean_dict[i].values
        full_matrix.loc[temp_index,i] = mean_dict[i].values

    receptors = cell_LR_nodes[cell_LR_nodes["category"]=="Receptor"]["identifier"].tolist()
    receptors = [i.split("_")[0] for i in receptors]
    receptors_matrix = matrix.loc[receptors]

    receptors_matrix = receptors_matrix[~receptors_matrix.index.duplicated(keep='first')]


    mean_dict = {}
    for i in cell_groups:
        cells = meta[meta["labels"]==i]["cell"].tolist()
        mean_dict[i] = receptors_matrix[cells].mean(axis=1)

    full_matrix = full_matrix[~full_matrix.index.duplicated(keep='first')]
    full_matrix = full_matrix.loc[:,~full_matrix.columns.duplicated()].copy()


    for i in mean_dict.keys():
        temp_index = [i+"_Receptor" for i in mean_dict[i].index.tolist()]
        full_matrix.loc[i,temp_index] = mean_dict[i].values
        full_matrix.loc[temp_index,i] = mean_dict[i].values

    #full_matrix = (full_matrix - np.min(full_matrix)) / (np.max(full_matrix) - np.min(full_matrix))

    full_matrix.values[np.where(np.isnan(full_matrix.values))] = 0
    full_matrix.values[np.where(np.isinf(full_matrix.values))] = 0

    cell_LR_data.x = torch.Tensor(full_matrix.values)

    LR_ids = cell_LR_nodes[(cell_LR_nodes["category"]=="Ligand") | (cell_LR_nodes["category"]=="Receptor")]["Id"].tolist()

    # true_values["Src"] = [i + "_Ligand" for i in true_values["Src"].tolist()]
    # true_values["Dst"] = [i + "_Receptor" for i in true_values["Dst"].tolist()]

    cell_groups = cell_LR_nodes[cell_LR_nodes["category"]=="Cell Group"]['identifier'].tolist()

    truth_list = []
    for i in cell_LR_nodes["identifier"].tolist():
        if "Ligand" in i:
            if i in ident_interactions["Src"].tolist():
                #truth_list.append(ident_interactions[ident_interactions["Src"]==i].index.tolist()[0] + 1)
                truth_list.append(1)
            else:
                truth_list.append(0)
        elif "Receptor" in i:
            if i in ident_interactions["Dst"].tolist():
                #truth_list.append(ident_interactions[ident_interactions["Dst"]==i].index.tolist()[0] + 1)
                truth_list.append(1)
            else:
                truth_list.append(0)
        else:
            truth_list.append(2)

    cell_LR_data.y = torch.Tensor(truth_list).type(torch.LongTensor)

    truth_array = np.array(truth_list)
    positive_classes = np.where(truth_array==1)[0].tolist()
    negative_classes = np.where(truth_array==0)[0].tolist()[:len(positive_classes)]

    new_train_mask = np.array([False]*truth_array.shape[0])
    new_train_mask[positive_classes + negative_classes] = True

    model = GAT(cell_LR_data,num_classes=2).to(device)
    data = cell_LR_data.to(device)
    data.train_mask = torch.Tensor(new_train_mask).type(torch.LongTensor)

    optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
    criterion = torch.nn.CrossEntropyLoss()

    ligands = cell_LR_nodes[cell_LR_nodes["category"]=="Ligand"]["Id"].tolist()
    receptors = cell_LR_nodes[cell_LR_nodes["category"]=="Receptor"]["Id"].tolist()

    truth_df = full_matrix.loc[cell_LR_nodes[cell_LR_nodes["category"]=="Ligand"]["identifier"].tolist(),cell_LR_nodes[cell_LR_nodes["category"]=="Receptor"]["identifier"].tolist()]

    truth_Tensor = torch.Tensor(truth_df.values).to(device)

    for epoch in range(1):
        model.train()
        optimizer.zero_grad()
        out = model(data)
        #loss = F.nll_loss(out[new_train_mask],data.y[new_train_mask])
        ligand_out = out[ligands,:]
        receptor_out = out[receptors,:]
        total_out = torch.inner(ligand_out,receptor_out)
        #loss = criterion(out[LR_ids],data.y)
        #loss = criterion(total_out,truth_Tensor)
        loss = criterion(out[new_train_mask],data.y[new_train_mask])
        loss.backward()
        optimizer.step()


    model.eval()
    cell_LR_out = model(data)

    ligands = cell_LR_nodes[cell_LR_nodes["category"]=="Ligand"]["Id"].tolist()
    receptors = cell_LR_nodes[cell_LR_nodes["category"]=="Receptor"]["Id"].tolist()

    ligand_out = cell_LR_out[ligands,:]
    receptor_out = cell_LR_out[receptors,:]
    _,ligand_pred = ligand_out.max(dim=1)
    _,receptor_pred = receptor_out.max(dim=1)

    total_out = torch.inner(ligand_out,receptor_out).cpu().detach().numpy()
    
    cell_LR_nodes.index = cell_LR_nodes["Id"].tolist()


    ligands_df = cell_LR_nodes[cell_LR_nodes["category"]=="Ligand"]
    receptors_df = cell_LR_nodes[cell_LR_nodes["category"]=="Receptor"]
    ligands_df.index = range(ligands_df.shape[0])
    receptors_df.index = range(receptors_df.shape[0])

    valid_ligands = ligands_df
    valid_receptors = receptors_df
    
    ligand_pred = ligand_pred.cpu().detach().numpy()
    receptor_pred = receptor_pred.cpu().detach().numpy()
    
    cell_LR_nodes.index = cell_LR_nodes["Id"].tolist()
#     valid_ligands = cell_LR_nodes.loc[np.where(ligand_pred == 1)]
#     valid_receptors = cell_LR_nodes.loc[np.where(receptor_pred == 1)]
#     ligand_out = ligand_out[np.where(ligand_pred ==1)]
#     receptor_out = receptor_out[np.where(receptor_pred ==1)]  
    total_out = torch.inner(ligand_out,receptor_out).cpu().detach().numpy()
    ligand_nodes = cell_LR_nodes[cell_LR_nodes["category"] == "Ligand"]
    ligand_nodes.index = range(0,ligand_nodes.shape[0])
    ligand_idents = ligand_nodes.iloc[np.where(ligand_pred==1)]['identifier'].tolist()
    total_out_df = pd.DataFrame(total_out,index=valid_ligands["identifier"].tolist(),columns=valid_receptors["identifier"].tolist())
    indicies = np.where(total_out_df.values > 0)
    source = list(indicies[0])
    dest = list(indicies[1])
    index_df = pd.DataFrame({"Id":range(0,total_out_df.shape[0]),"identifier":total_out_df.index.tolist()})
    column_df = pd.DataFrame({"Id":range(0,total_out_df.shape[1]),"identifier":total_out_df.columns.tolist()})
    source_list = index_df.loc[source]["identifier"].tolist()
    dest_list = column_df.loc[dest]["identifier"].tolist()
    total_link_df = pd.DataFrame({"Src":source_list,"Dst":dest_list,"Prob":total_out_df.values[indicies]})
    total_link_df = total_link_df.sort_values("Prob",ascending=False)
    Omnipath_db = pd.read_csv("/h/soemily/GAT/data/LR_database/Omnipath_database.csv")
    total_link_df["Src"] = [i.split("_")[0] for i in total_link_df["Src"].tolist()]
    total_link_df["Dst"] = [i.split("_")[0] for i in total_link_df["Dst"].tolist()]
    total_link_df = total_link_df.drop_duplicates()
    return total_link_df


In [91]:
matrix = pd.read_csv("/h/soemily/GAT/data/Pre_Post/GSE150949/pc9_t7_rep1.csv",index_col=0)
first_matrix,first_meta,first_nodes,first_interactions,first_LR_nodes,first_Omnipath_network = make_nodes_interactions(matrix)
first_LR_nodes.index = first_LR_nodes["Id"].tolist()
first_Omnipath_network["Src"] = first_LR_nodes.loc[first_Omnipath_network["Src"].tolist()]["identifier"].tolist()
first_Omnipath_network["Dst"] = first_LR_nodes.loc[first_Omnipath_network["Dst"].tolist()]["identifier"].tolist()
matrix = pd.read_csv("/h/soemily/GAT/data/Pre_Post/GSE150949/pc9_t7_rep2.csv",index_col=0)
second_matrix,second_meta,second_nodes,second_interactions,second_LR_nodes,second_Omnipath_network = make_nodes_interactions(matrix)
second_LR_nodes.index = second_LR_nodes["Id"].tolist()
second_Omnipath_network["Src"] = second_LR_nodes.loc[second_Omnipath_network["Src"].tolist()]["identifier"].tolist()
second_Omnipath_network["Dst"] = second_LR_nodes.loc[second_Omnipath_network["Dst"].tolist()]["identifier"].tolist()

         Falling back to preprocessing with `sc.pp.pca` and default params.
         Falling back to preprocessing with `sc.pp.pca` and default params.


In [98]:
rep_1_list = []
rep_2_list = []
for i in range(10):
    LR_nodes = pd.concat([first_LR_nodes,second_LR_nodes]).drop_duplicates("identifier")
    LR_nodes["Id"] = range(total_LR_nodes.shape[0])
    Omnipath_network = pd.concat([first_Omnipath_network,second_Omnipath_network]).drop_duplicates(["Src","Dst"])
    LR_nodes.index = LR_nodes["identifier"].tolist()
    Omnipath_network["Src"] = LR_nodes.loc[Omnipath_network["Src"].tolist()]["Id"].tolist()
    Omnipath_network["Dst"] = LR_nodes.loc[Omnipath_network["Dst"].tolist()]["Id"].tolist()
    Omnipath_network["Src"] = Omnipath_network["Src"].sample(frac=1).tolist()
    Omnipath_network["Dst"] = Omnipath_network["Dst"].sample(frac=1).tolist()
    print("Done Preprocessing")
    df = get_Omnipath_embeddings(LR_nodes,Omnipath_network)
    print("Done Omnipath")
    LR_nodes.index = LR_nodes["identifier"].tolist()
    Omnipath_network["Src"] = LR_nodes.loc[Omnipath_network["Src"].tolist()]["Id"].tolist()
    Omnipath_network["Dst"] = LR_nodes.loc[Omnipath_network["Dst"].tolist()]["Id"].tolist()
    Omnipath_data,Omnipath_nodes,Omnipath_interactions = make_dataset(LR_nodes,Omnipath_network,first=False,pathway_encode=False)
    Omnipath_nodes.index = Omnipath_nodes["identifier"].tolist()
    first_interactions["Src"] = first_interactions["Src"].sample(frac=1).tolist()
    first_interactions["Dst"] = first_interactions["Dst"].sample(frac=1).tolist()  
    second_interactions["Src"] = second_interactions["Src"].sample(frac=1).tolist()
    second_interactions["Dst"] = second_interactions["Dst"].sample(frac=1).tolist()  
    first_genes = list(set(first_matrix.index.tolist())& set(Omnipath_nodes.index.tolist()))
    second_genes = list(set(second_matrix.index.tolist())& set(Omnipath_nodes.index.tolist()))
    first_Omnipath_nodes = Omnipath_nodes.loc[first_genes]
    second_Omnipath_nodes = Omnipath_nodes.loc[second_genes]
    first_Omnipath_nodes["Id"] = range(first_Omnipath_nodes.shape[0])
    second_Omnipath_nodes["Id"] = range(second_Omnipath_nodes.shape[0])
    first_Omnipath_nodes.index = first_Omnipath_nodes["identifier"].tolist()
    second_Omnipath_nodes.index = second_Omnipath_nodes["identifier"].tolist()
    first_Omnipath_interactions = Omnipath_interactions[(Omnipath_interactions["Src"].isin(first_Omnipath_nodes["identifier"].tolist())) & (Omnipath_interactions["Dst"].isin(first_Omnipath_nodes["identifier"].tolist()))]
    second_Omnipath_interactions = Omnipath_interactions[(Omnipath_interactions["Src"].isin(second_Omnipath_nodes["identifier"].tolist())) & (Omnipath_interactions["Dst"].isin(second_Omnipath_nodes["identifier"].tolist()))]
    rep_1_list.append(get_cell_LR_embeddings(first_matrix,first_meta,first_nodes,first_interactions,df,first_Omnipath_nodes,first_Omnipath_interactions))
    print("Done Rep 1")
    rep_2_list.append(get_cell_LR_embeddings(second_matrix,second_meta,second_nodes,second_interactions,df,second_Omnipath_nodes,second_Omnipath_interactions))
    print("Done Rep 2")

Done Preprocessing
Done Omnipath
